In [1]:
import pandas as pd
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

In [2]:
loss_des_df = pd.read_csv('hashed_cleaned_loss_desc_from_5_ipynb.csv',index_col=0)

In [3]:
# Changing dtype of hashed_loss_desc_padded column
loss_des_df['hashed_loss_desc_padded'] = loss_des_df['hashed_loss_desc_padded'].apply(lambda x: np.fromstring(x[1:-1],dtype='int32',sep=' '))

# Using Gensim's pre-trained word2vec to generate vectors

In [9]:
from gensim.models import KeyedVectors

In [10]:
model = KeyedVectors.load_word2vec_format('google_new_embedded_model.bin', binary=True) 

In [16]:
loss_des_df['LossDescription'].h.apply(lambda x: [model.wv[y] for y in x.split()])

/home/ambuj/izenda_fullcycle/environment/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'test job' not in vocabulary"

In [5]:
loss_des_df

,EstimateTotal,LossDescription,hashed_loss_desc,EstimateTotal_bins,hashed_loss_desc_len,hashed_loss_desc_padded,LossYearMo
2,5241.89,test job,"[9816, 5179]",g,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9816, 5179]",201712
12,5149.06,fire start master bedroom spoke homeown call b...,"[3692, 9350, 6087, 886, 9252, 4628, 1443, 689,...",g,10,"[0, 0, 0, 0, 3692, 9350, 6087, 886, 9252, 4628...",201405
20,211.52,ph cook thought turn stove actual turn burner ...,"[7331, 2165, 9874, 10167, 9452, 67, 10167, 137...",a,9,"[0, 0, 0, 0, 0, 7331, 2165, 9874, 10167, 9452,...",201410
21,5010.55,tree fell hous insul cloth h allerg reaction,"[10089, 3623, 4675, 4944, 1914, 4345, 250, 7938]",g,8,"[0, 0, 0, 0, 0, 0, 10089, 3623, 4675, 4944, 19...",201410
25,11523.38,fire due candl upstair bonu room garag,"[3692, 3048, 1476, 10342, 1115, 8394, 4038]",i,7,"[0, 0, 0, 0, 0, 0, 0, 3692, 3048, 1476, 10342,...",201412
26,895.13,h prefer brother assist,"[4345, 7607, 1281, 564]",b,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4345, 7607, 128...",201501
28,1759.14,fire loss appear downstair total loss upstair ...,"[3692, 5835, 417, 2958, 10018, 5835, 10342, 63...",d,10,"[0, 0, 0, 0, 3692, 5835, 417, 2958, 10018, 583...",201502
31,279.12,textil bedroom closet dresser etc,"[9821, 886, 1911, 2985, 3383]",a,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 9821, 886, 1911, 2...",201505
32,20949.99,remov textil insur enorm stuff anim collect re...,"[8114, 9821, 4949, 3287, 9509, 354, 1978, 8114...",j,9,"[0, 0, 0, 0, 0, 8114, 9821, 4949, 3287, 9509, ...",201505
34,361.22,tree fell hous appear textil need clean pack,"[10089, 3623, 4675, 417, 9821, 6681, 1864, 7082]",a,8,"[0, 0, 0, 0, 0, 0, 10089, 3623, 4675, 417, 982...",201506


# Splitting Data

In [4]:
test2017=loss_des_df[loss_des_df["LossYearMo"].apply(lambda x:x>=201701)]

In [5]:
learning=loss_des_df[loss_des_df["LossYearMo"].apply(lambda x:x<201701)]

In [6]:
X = learning['hashed_loss_desc_padded']

In [7]:
loss_des_df['hashed_loss_desc_padded'][2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 9816, 5179], dtype=int32)

In [8]:
y = learning['EstimateTotal_bins']

In [9]:
y_dummies = pd.get_dummies(y,sparse=True)

In [10]:
y_dummies.head()

,a,b,c,d,e,f,g,h,i,j
12,0,0,0,0,0,0,1,0,0,0
20,1,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,1,0,0,0
25,0,0,0,0,0,0,0,0,1,0
26,0,1,0,0,0,0,0,0,0,0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y_dummies, test_size=0.33, random_state=42)

In [12]:
X_train.shape, y_train.shape

((335176,), (335176, 10))

In [13]:
X_val.shape, y_val.shape

((165087,), (165087, 10))

In [14]:
test2017_X = test2017['hashed_loss_desc_padded']

In [15]:
test2017_y = test2017['EstimateTotal_bins']

In [16]:
test2017_y_dummies = pd.get_dummies(test2017_y,sparse=True)

In [17]:
test2017_y_dummies.head()

,a,b,c,d,e,f,g,h,i,j
2,0,0,0,0,0,0,1,0,0,0
59,0,0,0,0,0,0,0,1,0,0
663,0,0,0,0,1,0,0,0,0,0
664,0,0,0,0,0,0,0,1,0,0
667,0,0,0,0,0,1,0,0,0,0


# Building the LSTM Model

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [19]:
X_train_temp  = X_train
y_train_temp = y_train
X_val_temp = X_val
y_val_temp = y_val

In [20]:
X_train_temp = np.stack(X_train_temp, axis=0 )
X_val_temp = np.stack(X_val_temp, axis=0 )

In [21]:
X_train_temp.shape, y_train_temp.shape, X_val_temp.shape, y_val_temp.shape

((335176, 14), (335176, 10), (165087, 14), (165087, 10))

In [22]:
pad_length = 14

#### Creating the model

In [23]:
# create the model
embedding_vecor_length = 300
model = Sequential()
model.add(Embedding(11024, embedding_vecor_length, input_length=pad_length))
model.add(LSTM(10))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
print(model.summary())
model.fit(X_train_temp, y_train_temp.values, validation_data=(X_val_temp,y_val_temp.values), callbacks=[early_stop],epochs=50, batch_size=128,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 14, 300)           3307200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                12440     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
Total params: 3,319,750
Trainable params: 3,319,750
Non-trainable params: 0
_________________________________________________________________
None
Train on 335176 samples, validate on 165087 samples
Epoch 1/50
335176/335176 [==============================] - 66s 196us/step - loss: 2.2192 - acc: 0.1663 - val_loss: 2.2009 - val_acc: 0.1735
Epoch 2/50
335176/335176 [==============================] - 65s 193us/step - loss: 2.1746 - acc: 0.1903 - val_loss: 2.2019 - val_acc: 0.1740
Epoch 3/50
335176/335176 [==============================]

In [24]:
model.predict(np.stack(test2017_X.head(),axis=0))

array([[0.15595298, 0.10751417, 0.08130127, 0.08520517, 0.08001376,
        0.09461036, 0.09196792, 0.11253645, 0.10462879, 0.08626921],
       [0.05209599, 0.07098914, 0.07051239, 0.06146665, 0.0674176 ,
        0.07000018, 0.08985337, 0.10359062, 0.15242027, 0.26165378],
       [0.09763595, 0.06364504, 0.06252315, 0.06579589, 0.07367233,
        0.07757444, 0.10646829, 0.1445108 , 0.17144212, 0.13673201],
       [0.08444022, 0.14115989, 0.12586512, 0.11177254, 0.09638476,
        0.09267557, 0.08648941, 0.08808114, 0.08501653, 0.08811477],
       [0.09248487, 0.06003279, 0.06106327, 0.06527528, 0.06933351,
        0.07683233, 0.1086968 , 0.15052757, 0.172159  , 0.14359458]],
      dtype=float32)

In [25]:
test2017_preds = model.predict_classes(np.stack(test2017_X,axis=0))

In [26]:
test2017_final_df = pd.DataFrame()

In [27]:
test2017_final_df['actual_y'] = test2017_y

In [28]:
test2017_final_df['preds'] = test2017_preds

In [29]:
test2017_final_df

,actual_y,preds
2,g,0
59,h,9
663,e,8
664,h,1
667,f,8
668,i,0
669,f,8
670,g,8
671,j,8
673,d,8


In [30]:
mapping_dict = {0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'i',9:'j'}

In [31]:
test2017_final_df['new_preds'] = test2017_final_df['preds'].map(mapping_dict)

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy_score(test2017_final_df['actual_y'], test2017_final_df['new_preds'])

0.16311409194236223

#### Creating the model

In [34]:
# create the model2
embedding_vecor_length = 100
model2 = Sequential()
model2.add(Embedding(11024, embedding_vecor_length, input_length=pad_length))
model2.add(LSTM(15))
model2.add(Dense(y_train.shape[1], activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
print(model2.summary())
model2.fit(X_train_temp, y_train_temp.values, validation_data=(X_val_temp,y_val_temp.values), callbacks=[early_stop],epochs=50, batch_size=128,verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 14, 100)           1102400   
_________________________________________________________________
lstm_2 (LSTM)                (None, 15)                6960      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                160       
Total params: 1,109,520
Trainable params: 1,109,520
Non-trainable params: 0
_________________________________________________________________
None
Train on 335176 samples, validate on 165087 samples
Epoch 1/50
109824/335176 [========>.....................] - ETA: 31s - loss: 2.2393 - acc: 0.1552

KeyboardInterrupt: 

In [ ]:
accuracy_score(test2017_final_df['actual_y'], pd.Series(model2.predict_classes(np.stack(test2017_X,axis=0))).map(mapping_dict))